In [1]:
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
import torch.optim
import pytorch_forecasting

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore')

2022-11-30 15:30:24.988290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 15:30:26.994494: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda-11.7/lib64:
2022-11-30 15:30:26.998136: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda-11.7/lib64:
2022-11-30 15:30:26.998182: W tensorflow/compiler/tf2tensorrt/utils/py_utils.c

In [2]:
csv = pd.read_csv("2016_2022.csv")
data = csv[lambda x : x.Source == 3][:10000]
data["time_idx"] = [i for i in range(len(data))]

In [3]:
data

,Date,Valeur,Source,time_idx
1,01/01/2016 00:00:00,3.607,3,0
13,01/01/2016 00:10:00,3.496,3,1
25,01/01/2016 00:20:00,3.446,3,2
37,01/01/2016 00:30:00,3.298,3,3
49,01/01/2016 00:40:00,3.229,3,4
...,...,...,...,...
119853,10/03/2016 10:10:00,0.700,3,9995
119865,10/03/2016 10:20:00,0.508,3,9996
119877,10/03/2016 10:30:00,0.425,3,9997
119889,10/03/2016 10:40:00,0.283,3,9998


In [4]:
max_prediction_length = 5
max_encoder_length = 50
training_cutoff = data["time_idx"].max() - max_prediction_length

In [5]:
training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Valeur",
    group_ids=["Source"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["Date"],
    static_reals=["Source"],
    time_varying_known_categoricals=["Date"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=["Valeur"],
    categorical_encoders = {'Date' : pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True, warn = False)},
    #target_normalizer=GroupNormalizer(groups=["agency", "sku"], transformation="softplus"),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

In [6]:
training

TimeSeriesDataSet[length=9999](
	time_idx='time_idx',
	target='Valeur',
	group_ids=['Source'],
	weight=None,
	max_encoder_length=50,
	min_encoder_length=25,
	min_prediction_idx=0,
	min_prediction_length=1,
	max_prediction_length=5,
	static_categoricals=['Date'],
	static_reals=['Source', 'encoder_length', 'Valeur_center', 'Valeur_scale'],
	time_varying_known_categoricals=['Date'],
	time_varying_known_reals=['time_idx', 'relative_time_idx'],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['Valeur'],
	variable_groups={},
	constant_fill_strategy={},
	allow_missing_timesteps=False,
	lags={},
	add_relative_time_idx=True,
	add_target_scales=True,
	add_encoder_length=True,
	target_normalizer=EncoderNormalizer(
	method='standard',
	center=True,
	max_length=None,
	transformation='relu',
	method_kwargs={}
),
	categorical_encoders={'Date': NaNLabelEncoder(add_nan=True, warn=False), '__group_id__Source': NaNLabelEncoder(add_nan=False, warn=True)},
	scalers={'Source': StandardSca

In [7]:
# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [8]:
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

0.4189999997615814

In [9]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
    accelerator = 'gpu',
    devices = 1,
    max_epochs = 50,
)

Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=4,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=4,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=2,  # set to <= hidden_size
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Number of parameters in network: 42.0k


In [11]:
# find optimal learning rate
res = trainer.tuner.lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Restoring states from the checkpoint path at /mnt/c/Users/alex4/Desktop/COURS/PIIA-Marees/PyTorch/.lr_find_84be5a1f-b284-4384-ae0b-a23ad1d6d536.ckpt


KeyError: 'radam_buffer'

In [13]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=30,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
    auto_lr_find = False,
    accelerator = 'gpu',
    devices = 1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=4,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=2,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=1,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 42.0k


In [14]:
# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 40.0 K
3  | prescalers                         | ModuleDict                      | 28    
4  | static_variable_selection          | VariableSelectionNetwork        | 399   
5  | encoder_variable_selection         | VariableSelectionNetwork        | 332   
6  | decoder_variable_selection         | VariableSelectionNetwork        | 222   
7  | static_context_variable_selection  | GatedResidualNetwork            | 88    
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 88    
9  | static_context_initial_cell_lstm 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.


AttributeError: module 'tensorflow._api.v2.io.gfile' has no attribute 'get_filesystem'